In [1]:
#%matplotlib notebook
%matplotlib inline 

%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd


sys.path.append("../../pycode/")
#from magus_util import read_nvprof_trace, parse_nvprof_trace, getruntime, sort_dict_by_val, genNNFeat, testFeat
from magus_util import *
from magus_contention import *

# load dedicated execution time

In [2]:
dedicated_app_dd = np.load("../../09_case_study/dedicated_app_runtime.npy").item()

In [3]:
# test 
dedicated_app_dd['rodinia_b+tree']

2642.2

# load the app2metric 

In [4]:
app2metric_dd = np.load("../../07_sim_devid/similarity/app2metric_dd.npy").item()

In [5]:
# test
app2metric_dd['rodinia_b+tree']

branch_efficiency                                1
warp_execution_efficiency                 0.968361
warp_nonpred_execution_efficiency         0.953817
local_store_transactions_per_request             0
gld_transactions_per_request              0.180095
gst_transactions_per_request               0.03125
global_hit_rate                           0.687571
local_hit_rate                                   0
gst_requested_throughput                0.00274652
gld_throughput                            0.389003
local_memory_overhead                            0
tex_cache_hit_rate                        0.625726
tex_cache_throughput                      0.276062
l2_tex_write_throughput                  0.0070407
sysmem_write_throughput                          0
shared_store_throughput                          0
gst_efficiency                               0.225
stall_inst_fetch                         0.0407055
stall_exec_dependency                    0.0354754
stall_memory_dependency        

# similarity distance

In [6]:
#
# compute similarity distance
#
app2app_sim_dd = {}
for app1, app1Metric in app2metric_dd.iteritems():
    app1_mat = app1Metric.as_matrix()
    app1_dist_dd = {}
    for app2, app2Metric in app2metric_dd.iteritems():
        if app1 <> app2:
            app2_mat = app2Metric.as_matrix()
            dist = np.linalg.norm(app1_mat - app2_mat)
            app1_dist_dd[app2] = dist
    app2app_sim_dd[app1] =  app1_dist_dd
    

In [7]:
# test
app2app_sim_dd['rodinia_b+tree']

{'cudasdk_BlackScholes': 1.4433313176453999,
 'cudasdk_FDTD3d': 0.98713237056515335,
 'cudasdk_MCEstimatePiInlineP': 1.6920907324126799,
 'cudasdk_MCEstimatePiInlineQ': 2.0195519909691804,
 'cudasdk_MCEstimatePiP': 1.504088427974352,
 'cudasdk_MCEstimatePiQ': 1.8821228802671122,
 'cudasdk_MCSingleAsianOptionP': 2.0963150611532888,
 'cudasdk_SobolQRNG': 1.7089935914544723,
 'cudasdk_batchCUBLAS': 1.8387153155621019,
 'cudasdk_binomialOptions': 1.4333869284563876,
 'cudasdk_boxFilterNPP': 1.6031474959434,
 'cudasdk_c++11Cuda': 0.99212749566055125,
 'cudasdk_concurrentKernels': 1.8021618829339294,
 'cudasdk_convolutionFFT2D': 1.7874343326242677,
 'cudasdk_convolutionSeparable': 1.3716535302507027,
 'cudasdk_convolutionTexture': 1.8247456264132378,
 'cudasdk_dct8x8': 1.7358171004361904,
 'cudasdk_dwtHaar1D': 1.6917669538328202,
 'cudasdk_dxtc': 1.0052477743549133,
 'cudasdk_eigenvalues': 1.3132675102326719,
 'cudasdk_fastWalshTransform': 1.3944322006436458,
 'cudasdk_interval': 2.145503886

In [8]:
#
# save
#
np.save('app2app_sim_dd.npy', app2app_sim_dd)

# extract feature from traces

In [9]:
target_traces = [
'../../07_sim_devid/similarity/traces/traces_cudasdk_batchCUBLAS.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_binomialOptions.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_BlackScholes.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_boxFilterNPP.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_c++11Cuda.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_concurrentKernels.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_convolutionFFT2D.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_convolutionSeparable.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_convolutionTexture.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_dct8x8.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_dwtHaar1D.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_dxtc.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_eigenvalues.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_fastWalshTransform.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_FDTD3d.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_interval.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_lineOfSight.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_matrixMul.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_MCEstimatePiInlineP.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_MCEstimatePiInlineQ.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_MCEstimatePiP.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_MCEstimatePiQ.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_MCSingleAsianOptionP.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_mergeSort.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_quasirandomGenerator.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_radixSortThrust.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_reduction.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_scalarProd.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_scan.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_segmentationTreeThrust.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_shflscan.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_simpleCUBLAS.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_simpleCUFFTcallback.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_SobolQRNG.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_sortingNetworks.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_stereoDisparity.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_threadFenceReduction.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_transpose.csv',
'../../07_sim_devid/similarity/traces/traces_cudasdk_vectorAdd.csv',
'../../07_sim_devid/similarity/traces/traces_lonestar_bh.csv',
'../../07_sim_devid/similarity/traces/traces_lonestar_dmr.csv',
'../../07_sim_devid/similarity/traces/traces_lonestar_mst.csv',
'../../07_sim_devid/similarity/traces/traces_lonestar_sssp.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_bfs.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_cutcp.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_lbm.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_mriq.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_sgemm.csv',
'../../07_sim_devid/similarity/traces/traces_parboil_stencil.csv',
'../../07_sim_devid/similarity/traces/traces_poly_2dconv.csv',
'../../07_sim_devid/similarity/traces/traces_poly_3dconv.csv',
'../../07_sim_devid/similarity/traces/traces_poly_3mm.csv',
'../../07_sim_devid/similarity/traces/traces_poly_atax.csv',
'../../07_sim_devid/similarity/traces/traces_poly_bicg.csv',
'../../07_sim_devid/similarity/traces/traces_poly_correlation.csv',
'../../07_sim_devid/similarity/traces/traces_poly_covariance.csv',
'../../07_sim_devid/similarity/traces/traces_poly_fdtd2d.csv',
'../../07_sim_devid/similarity/traces/traces_poly_gemm.csv',
'../../07_sim_devid/similarity/traces/traces_poly_gesummv.csv',
'../../07_sim_devid/similarity/traces/traces_poly_mvt.csv',
'../../07_sim_devid/similarity/traces/traces_poly_syr2k.csv',
'../../07_sim_devid/similarity/traces/traces_poly_syrk.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_backprop.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_b+tree.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_dwt2d.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_gaussian.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_heartwall.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_hotspot.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_hybridsort.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_lavaMD.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_lud.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_needle.csv',
'../../07_sim_devid/similarity/traces/traces_rodinia_pathfinder.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1BFS.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1fft.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1GEMM.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1md5hash.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1reduction.csv',
'../../07_sim_devid/similarity/traces/traces_shoc_lev1sort.csv']

In [10]:
#=====================================================#
#   test
#=====================================================#
print target_traces[0]
df_trace = read_nvprof_trace(target_traces[0])
# df_trace

# for each api
# [apiType, startT, endT, Grid, Block, Reg, SharedMem, tranSize]
appTraceList = parse_nvprof_trace(df_trace, rowinfoVer=1)
# appTraceList

#=======================#
# whether iobound or kernelbound
#=======================#
ioTime = 0.
kernTime = 0.
for appInfo in appTraceList:
    [apiType, startT, endT, Grid, Block, Reg, SharedMem, tranSize] = appInfo
    if apiType == 'kern':
        kernTime += endT - startT
    else:
        ioTime += endT - startT
        
#print ioTime, kernTime

ioBound = 0
if ioTime > kernTime:
    ioBound = 1

if ioBound == 0:
    print "kernel bound"
else:
    print "io bound"
    
#=======================#
# extract trace feature
#=======================#
feat_list = testFeat_v1(appTraceList)

[appDur, prepareTime, gpuTime, threads_max, threads_avg, reg_max, reg_avg, sm_max, sm_avg, trans_max, trans_avg] = feat_list

print "appDur, prepareTime, gpuTime"
print appDur, prepareTime, gpuTime, "\n"
print "threads_max,reg_max, sm_max"
print threads_max,reg_max, sm_max, "\n"

../../07_sim_devid/similarity/traces/traces_cudasdk_batchCUBLAS.csv
io bound
appDur, prepareTime, gpuTime
1246.937464 1176.737 70.200464 

threads_max,reg_max, sm_max
4096.0 240.0 16500.0 



In [11]:
app2newfeat_dd = {}
app_iobound_dd = {}

for appTrace_csv in target_traces:
    #print appTrace_csv
    
    appName = appTrace_csv[:-4][44:]
    print appName
    
    df_trace = read_nvprof_trace(appTrace_csv)  # read raw trace
    appTraceList = parse_nvprof_trace(df_trace, rowinfoVer=1)  # extract trace info from raw input
    
    #------------------------------------------#
    # decide whether io-bound or kernel-bound
    #------------------------------------------#
    ioTime = 0.
    kernTime = 0.
    for appInfo in appTraceList:
        [apiType, startT, endT, Grid, Block, Reg, SharedMem, tranSize] = appInfo
        if apiType == 'kern':
            kernTime += endT - startT
        else:
            ioTime += endT - startT

    #print ioTime, kernTime

    ioBound = 0
    if ioTime > kernTime:
        ioBound = 1
        
    app_iobound_dd[appName] = ioBound
    
    
    #------------------------------------------#
    # extract features from the traces
    #------------------------------------------#   
    feat_list = testFeat_v1(appTraceList)
    app2newfeat_dd[appName] = feat_list
    

cudasdk_batchCUBLAS
cudasdk_binomialOptions
cudasdk_BlackScholes
cudasdk_boxFilterNPP
cudasdk_c++11Cuda
cudasdk_concurrentKernels
cudasdk_convolutionFFT2D
cudasdk_convolutionSeparable
cudasdk_convolutionTexture
cudasdk_dct8x8
cudasdk_dwtHaar1D
cudasdk_dxtc
cudasdk_eigenvalues
cudasdk_fastWalshTransform
cudasdk_FDTD3d
cudasdk_interval
cudasdk_lineOfSight
cudasdk_matrixMul
cudasdk_MCEstimatePiInlineP
cudasdk_MCEstimatePiInlineQ
cudasdk_MCEstimatePiP
cudasdk_MCEstimatePiQ
cudasdk_MCSingleAsianOptionP
cudasdk_mergeSort
cudasdk_quasirandomGenerator
cudasdk_radixSortThrust
cudasdk_reduction
cudasdk_scalarProd
cudasdk_scan
cudasdk_segmentationTreeThrust
cudasdk_shflscan
cudasdk_simpleCUBLAS
cudasdk_simpleCUFFTcallback
cudasdk_SobolQRNG
cudasdk_sortingNetworks
cudasdk_stereoDisparity
cudasdk_threadFenceReduction
cudasdk_transpose
cudasdk_vectorAdd
lonestar_bh
lonestar_dmr
lonestar_mst
lonestar_sssp
parboil_bfs
parboil_cutcp
parboil_lbm
parboil_mriq
parboil_sgemm
parboil_stencil
poly_2dconv
pol

In [12]:
app_iobound_dd

{'cudasdk_BlackScholes': 1,
 'cudasdk_FDTD3d': 1,
 'cudasdk_MCEstimatePiInlineP': 0,
 'cudasdk_MCEstimatePiInlineQ': 0,
 'cudasdk_MCEstimatePiP': 0,
 'cudasdk_MCEstimatePiQ': 1,
 'cudasdk_MCSingleAsianOptionP': 0,
 'cudasdk_SobolQRNG': 1,
 'cudasdk_batchCUBLAS': 1,
 'cudasdk_binomialOptions': 0,
 'cudasdk_boxFilterNPP': 1,
 'cudasdk_c++11Cuda': 0,
 'cudasdk_concurrentKernels': 0,
 'cudasdk_convolutionFFT2D': 1,
 'cudasdk_convolutionSeparable': 1,
 'cudasdk_convolutionTexture': 1,
 'cudasdk_dct8x8': 1,
 'cudasdk_dwtHaar1D': 1,
 'cudasdk_dxtc': 1,
 'cudasdk_eigenvalues': 0,
 'cudasdk_fastWalshTransform': 1,
 'cudasdk_interval': 1,
 'cudasdk_lineOfSight': 1,
 'cudasdk_matrixMul': 0,
 'cudasdk_mergeSort': 1,
 'cudasdk_quasirandomGenerator': 1,
 'cudasdk_radixSortThrust': 1,
 'cudasdk_reduction': 1,
 'cudasdk_scalarProd': 1,
 'cudasdk_scan': 1,
 'cudasdk_segmentationTreeThrust': 1,
 'cudasdk_shflscan': 1,
 'cudasdk_simpleCUBLAS': 1,
 'cudasdk_simpleCUFFTcallback': 0,
 'cudasdk_sortingNetwor

In [13]:
app2newfeat_dd['shoc_lev1sort']

[746.713523,
 634.351631,
 112.36189200000001,
 131072.0,
 55338.666666666664,
 27.0,
 15.166666666666666,
 4125.0,
 3036.4583333333335,
 1000.0,
 1000.0]

In [14]:
#
# save
#
np.save('app_iobound_dd.npy', app_iobound_dd)
np.save('app2newfeat_dd.npy', app2newfeat_dd)

In [15]:
# #
# # check appdur(i) <= app_prepare(k)
# #
# target_appdur  = app2newfeat_dd[TARGET_APP][0]
# target_prepare = app2newfeat_dd[TARGET_APP][1]
# #print current_appdur

# large_job_list = []
# small_job_list = []

# for key, value in app2newfeat_dd.iteritems():
#     if key <> TARGET_APP:
#         my_appdur  = value[0]
#         my_prepare = value[1]
#         #print my_appdur, my_prepare
#         if my_prepare >= target_appdur:
#             large_job_list.append(key)
        
#         if my_appdur <= target_prepare:
#             small_job_list.append(key)

# #
# # analyze
# #
# print "large job list"
# print len(large_job_list), "\n"

# print "small job list"
# print len(small_job_list), "\n"

### large job list

In [16]:
# device_threads_max = 28 * 2048

# currentApp_speedup_dd = app_run2_groundTruth_dd[TARGET_APP]

# print "Target APP: "
# print TARGET_APP
# [thread_max, _, reg_max, _, sm_max, _, trans_max, _] = app2newfeat_dd[TARGET_APP][3:]
# print thread_max, trans_max
# print "\n"


# left_threads_max = device_threads_max - thread_max


# large_app1_speedup_list = []
# large_app2_speedup_list = []

# large_job_maxthread_dd = {}

# for i in large_job_list:
#     k = i
#     if i == "rodinia_b+tree":
#         k = "rodinia_btree"
#     if i== "cudasdk_c++11Cuda":
#         k = "cudasdk_c11Cuda"
#     #print currentApp_speedup_dd[i]
#     [app1_spdup, app2_spdup, _]= currentApp_speedup_dd[k]
#     large_app1_speedup_list.append(app1_spdup)
#     large_app2_speedup_list.append(app2_spdup)
    
    
#     print i
#     [thread_max, _, reg_max, _, sm_max, _, trans_max, _] = app2newfeat_dd[i][3:]
#     print thread_max, trans_max
#     print app1_spdup, app2_spdup
#     print "\n"
    
#     #large_job_maxthread_dd[i] = thread_max
#     if thread_max <= left_threads_max:
#         large_job_maxthread_dd[i] = thread_max
    

# #
# # sort
# #
# print sorted(large_job_maxthread_dd.items(), key=operator.itemgetter(1))[-3:]

# large_app1_speedup_mat = np.matrix(large_app1_speedup_list)
# print np.amin(large_app1_speedup_mat), np.amax(large_app1_speedup_mat), np.mean(large_app1_speedup_mat), np.std(large_app1_speedup_mat)

# large_app2_speedup_mat = np.matrix(large_app2_speedup_list)
# print np.amin(large_app2_speedup_mat), np.amax(large_app2_speedup_mat), np.mean(large_app2_speedup_mat), np.std(large_app2_speedup_mat)

### small jobs list

In [17]:
# small_app1_speedup_list = []
# small_app2_speedup_list = []

# for i in small_job_list:
#     if i == "rodinia_b+tree":
#         i = "rodinia_btree"
#     if i== "cudasdk_c++11Cuda":
#         i = "cudasdk_c11Cuda"
        
#     [app1_spdup, app2_spdup, _]= currentApp_speedup_dd[i]
#     small_app1_speedup_list.append(app1_spdup)
#     small_app2_speedup_list.append(app2_spdup)
    
# small_app1_speedup_mat = np.matrix(small_app1_speedup_list)
# print np.amin(small_app1_speedup_mat), np.amax(small_app1_speedup_mat), np.mean(small_app1_speedup_mat), np.std(small_app1_speedup_mat)

# small_app2_speedup_mat = np.matrix(small_app2_speedup_list)
# print np.amin(small_app2_speedup_mat), np.amax(small_app2_speedup_mat), np.mean(small_app2_speedup_mat), np.std(small_app2_speedup_mat)